# Submit Nanoribbon Workchain

In [ ]:
%aiida

In [ ]:
# General imports
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from IPython.display import display, clear_output, HTML
import itertools

# AiiDA & AiiDA lab imports
from aiida.orm import Str,StructureData
from aiida.engine import calcfunction
from aiida.orm import SinglefileData
from aiidalab_widgets_base.utils import string_range_to_list, list_to_string_range

from nanoribbon.viewers import Smiles2GNRWidget
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget,BasicStructureEditor, StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget
from aiida.engine import submit

# Work Chains
NanoribbonWorkChain = WorkflowFactory('nanoribbon')

In [ ]:
structure_selector = StructureManagerWidget(importers=[
    ("SMILES2GNR", Smiles2GNRWidget()),
    ("Import from computer", StructureUploadWidget()),
    ("AiiDA database", StructureBrowserWidget()),
    ],editors = [("Edit structure", BasicStructureEditor())],
    storable=False,node_class='StructureData')
display(structure_selector)

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

pw_code_dropdown = CodeDropdown(input_plugin='quantumespresso.pw')
pp_code_dropdown = CodeDropdown(input_plugin='quantumespresso.pp')
projwfc_code_dropdown = CodeDropdown(input_plugin='quantumespresso.projwfc')

text_calc_description = ipw.Text(description='Calculation Name: ',
                          placeholder='A meaningful name',
                          style=style, layout=layout)
spin_u = ipw.Text(placeholder='1..10 15',
                                    description='IDs atoms spin U',
                                    style=style, layout={'width': '60%'})
spin_d = ipw.Text(placeholder='1..10 15',
                                    description='IDs atoms spin D',
                                    style=style, layout={'width': '60%'})
display(ipw.VBox([spin_u, spin_d, text_calc_description, pw_code_dropdown, pp_code_dropdown, projwfc_code_dropdown, ]))

In [ ]:
## spin of structure in strcture browser
def check_spins(atoms):
    tags = atoms.get_tags()
    su=np.where(tags == 1)[0].tolist()
    sd=np.where(tags == 2)[0].tolist()
    if len(su) == 0 and len(sd) == 0:
        add_name_old = '-no_spin'
    else:
        add_name_old = '-spin-' + str (abs(len(su) - len(sd)))
        
    su_str=list_to_string_range(su)
    sd_str=list_to_string_range(sd)
    #spin_d.value=sd_str
    #spin_u.value=su_str
    return add_name_old, su_str,sd_str


@calcfunction
def set_spins(NODE,U,D):
    
    su = U.value
    sd = D.value
    atoms=NODE.get_ase()
    tags = np.zeros(len(atoms))
    for u in  string_range_to_list(su)[0]:
        tags[u]=1
    for d in  string_range_to_list(sd)[0]:
        tags[d]=2    

    nu = len(string_range_to_list(su)[0])
    nd = len(string_range_to_list(sd)[0])
    if nu == 0 and nd  == 0:
        add_name = '-no_spin'
    else:
        add_name = '-spin-' + str(abs(nu - nd))
        
    atoms.set_tags(tags)

    new_struct = StructureData(ase=atoms)
    
    # ensure that tags got correctly translated into kinds 
    for t1, k in zip(tags, new_struct.get_site_kindnames()):
        t2 = int(k[-1]) if k[-1].isnumeric() else 0
        assert t1==t2
    
    #new_struct.store()
    old_name_split = NODE.description.split('spin')[0]
    if old_name_split == '':
        old_name_split = atoms.get_chemical_formula()    
    if old_name_split[-1] in ['_','-','.',',',' ']:
        old_name_split = old_name_split[:-1] 
    new_name = old_name_split + add_name    
    
    new_struct.description = new_name
    return new_struct

In [ ]:
## set initial spins according to tags of the structure
def on_structure_change(c=None):
    if structure_selector.structure:
        tags = structure_selector.structure.get_tags()
        su=np.where(tags == 1)[0].tolist()
        sd=np.where(tags == 2)[0].tolist()        
        su_str=list_to_string_range(su)
        sd_str=list_to_string_range(sd)
        spin_d.value=sd_str
        spin_u.value=su_str
        
structure_selector.observe(on_structure_change,names='structure')

In [ ]:
def on_submit():    
    ## CHECK spin consistency and create new structure in case mismatch
    atoms = structure_selector.structure #ASE atoms
    description = structure_selector.structure_node.description
    if description == '':
        description = structure_selector.structure.get_chemical_formula()
    ## spin from strucure browser
    add_name_old, su_str, sd_str = check_spins(atoms)
    ## spin in input widgets
    input_su = list_to_string_range(string_range_to_list(spin_u.value)[0])
    input_sd = list_to_string_range(string_range_to_list(spin_d.value)[0])
    spin_mismatch = (input_su != su_str and input_su != sd_str) or (input_sd != su_str and input_sd != sd_str)    
        
    with submit_out:
        clear_output()
        if not structure_selector.structure_node:
            print("Please select a structure.")
            return
        if not pw_code_dropdown.selected_code:
            print("Please select a pw code.")
            return
        if not pp_code_dropdown.selected_code:
            print("Please select a pp code.")
            return
        if not projwfc_code_dropdown.selected_code:
            print("Please select a projwfc code.")
            return        
        if len(text_calc_description.value) < 3:
            print("Please enter a calculation name.")
            return
        
        ## IF input spins do not match selected structure spin, create new structure
        if spin_mismatch :
            node_to_submit = set_spins(structure_selector.structure_node,Str(input_su),Str(input_sd))
            old_name_split = description.split('spin')[0]
            if old_name_split[-1] in ['_','-','.',',',' ']:
                old_name_split = old_name_split[:-1] 

            print("created new structure to match spin guess. PK: ", 
                  node_to_submit.pk, ' with name: ' + node_to_submit.description )
            structure_selector.structure_node.description = old_name_split + add_name_old
            print("renamed input PK: ", structure_selector.structure_node.pk,  ' in: ' + old_name_split + add_name_old )
        else:
            node_to_submit = structure_selector.structure_node        

        #pw_code, pp_code, projwfc_code = qe_code_groups[drop_codes.value]
        builder = NanoribbonWorkChain.get_builder()
        builder.pw_code = pw_code_dropdown.selected_code
        builder.pp_code = pp_code_dropdown.selected_code
        builder.projwfc_code = projwfc_code_dropdown.selected_code
        builder.structure = node_to_submit
        builder.precision = Float(1.0) #Float(slider_precision.value)
        builder.pseudo_family = Str('SSSP_modified') # Str('SSSP_precision_v1.0')
        builder.metadata = {
            "description": text_calc_description.value,
            "label": "NanoribbonWorkChain",
        }
        return builder

In [ ]:
btn_submit = SubmitButtonWidget(NanoribbonWorkChain,input_dictionary_function=on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)